In [2]:
# https://realpython.com/gradient-descent-algorithm-python/
# demo https://medium.com/code-heroku/gradient-descent-for-machine-learning-3d871fa48b4c
import numpy as np

def ssr_gradient(x, y, b):
    res = b[0] + b[1] * x - y
    return res.mean(), (res * x).mean()  # .mean() is a method of np.ndarray

def gradient_descent(
    gradient, x, y, start, learn_rate=0.1, n_iter=50, tolerance=1e-06,
    dtype="float64"
):
    # Checking if the gradient is callable
    if not callable(gradient):
        raise TypeError("'gradient' must be callable")

    # Setting up the data type for NumPy arrays
    dtype_ = np.dtype(dtype)

    # Converting x and y to NumPy arrays
    x, y = np.array(x, dtype=dtype_), np.array(y, dtype=dtype_)
    if x.shape[0] != y.shape[0]:
        raise ValueError("'x' and 'y' lengths do not match")

    # Initializing the values of the variables
    vector = np.array(start, dtype=dtype_)

    # Setting up and checking the learning rate
    learn_rate = np.array(learn_rate, dtype=dtype_)
    if np.any(learn_rate <= 0):
        raise ValueError("'learn_rate' must be greater than zero")

    # Setting up and checking the maximal number of iterations
    n_iter = int(n_iter)
    if n_iter <= 0:
        raise ValueError("'n_iter' must be greater than zero")

    # Setting up and checking the tolerance
    tolerance = np.array(tolerance, dtype=dtype_)
    if np.any(tolerance <= 0):
        raise ValueError("'tolerance' must be greater than zero")

    # Performing the gradient descent loop
    for _ in range(n_iter):
        # Recalculating the difference
        diff = -learn_rate * np.array(gradient(x, y, vector), dtype_)

        # Checking if the absolute difference is small enough
        if np.all(np.abs(diff) <= tolerance):
            break

        # Updating the values of the variables
        vector += diff

    return vector if vector.shape else vector.item()

def sgd(
    gradient, x, y, n_vars=None, start=None, learn_rate=0.1,
    decay_rate=0.0, batch_size=1, n_iter=50, tolerance=1e-06,
    dtype="float64", random_state=None
):
    # Checking if the gradient is callable
    if not callable(gradient):
        raise TypeError("'gradient' must be callable")

    # Setting up the data type for NumPy arrays
    dtype_ = np.dtype(dtype)

    # Converting x and y to NumPy arrays
    x, y = np.array(x, dtype=dtype_), np.array(y, dtype=dtype_)
    n_obs = x.shape[0]
    if n_obs != y.shape[0]:
        raise ValueError("'x' and 'y' lengths do not match")
    xy = np.c_[x.reshape(n_obs, -1), y.reshape(n_obs, 1)]

    # Initializing the random number generator
    seed = None if random_state is None else int(random_state)
    rng = np.random.default_rng(seed=seed)

    # Initializing the values of the variables
    vector = (
        rng.normal(size=int(n_vars)).astype(dtype_)
        if start is None else
        np.array(start, dtype=dtype_)
    )

    # Setting up and checking the learning rate
    learn_rate = np.array(learn_rate, dtype=dtype_)
    if np.any(learn_rate <= 0):
        raise ValueError("'learn_rate' must be greater than zero")

    # Setting up and checking the decay rate
    decay_rate = np.array(decay_rate, dtype=dtype_)
    if np.any(decay_rate < 0) or np.any(decay_rate > 1):
        raise ValueError("'decay_rate' must be between zero and one")

    # Setting up and checking the size of minibatches
    batch_size = int(batch_size)
    if not 0 < batch_size <= n_obs:
        raise ValueError(
            "'batch_size' must be greater than zero and less than "
            "or equal to the number of observations"
        )

    # Setting up and checking the maximal number of iterations
    n_iter = int(n_iter)
    if n_iter <= 0:
        raise ValueError("'n_iter' must be greater than zero")

    # Setting up and checking the tolerance
    tolerance = np.array(tolerance, dtype=dtype_)
    if np.any(tolerance <= 0):
        raise ValueError("'tolerance' must be greater than zero")

    # Setting the difference to zero for the first iteration
    diff = 0

    # Performing the gradient descent loop
    for _ in range(n_iter):
        # Shuffle x and y
        rng.shuffle(xy)

        # Performing minibatch moves
        for start in range(0, n_obs, batch_size):
            stop = start + batch_size
            x_batch, y_batch = xy[start:stop, :-1], xy[start:stop, -1:]

            # Recalculating the difference
            grad = np.array(gradient(x_batch, y_batch, vector), dtype_)
            diff = decay_rate * diff - learn_rate * grad

            # Checking if the absolute difference is small enough
            if np.all(np.abs(diff) <= tolerance):
                break

            # Updating the values of the variables
            vector += diff

    return vector if vector.shape else vector.item()

In [3]:
# Main
M = 512;  theta0 = np.pi/10; theta1 = np.pi*np.pi; 
X = np.random.rand(M,1)
sigma = .1
print('sigma =', sigma)
y = theta0 + theta1 * X + sigma*np.random.randn(M,1) 
print('theta0 = ', theta0, 'theta1 = ', theta1)
gradient_descent(ssr_gradient, X, y, start=[0.5, 0.5], learn_rate=0.0008,n_iter=100_000)

sigma = 0.1
theta0 =  0.3141592653589793 theta1 =  9.869604401089358


array([0.34130572, 9.82290327])

In [4]:
# sgd(ssr_gradient, X, y, start=[0.5, 0.5], learn_rate=0.0008, batch_size=3, n_iter=100_000, random_state=0)

In [5]:
sigma = 1
print('sigma =', sigma)
y = theta0 + theta1 * X + sigma*np.random.randn(M,1) 
gradient_descent(ssr_gradient, X, y, start=[0.5, 0.5], learn_rate=0.0008,n_iter=100_000)

sigma = 1


array([0.40832945, 9.63986039])

In [6]:
sigma = 10
print('sigma =', sigma)
y = theta0 + theta1 * X + sigma*np.random.randn(M,1) 
gradient_descent(ssr_gradient, X, y, start=[0.5, 0.5], learn_rate=0.0008,n_iter=100_000)

sigma = 10


array([-0.26942419, 10.54462299])

In [7]:
print('sigma =', sigma, 'learn rate =', 0.9)
gradient_descent(ssr_gradient, X, y, start=[0.5, 0.5], learn_rate=0.9,n_iter=100_000)

sigma = 10 learn rate = 0.9


array([-0.30034897, 10.60267075])

In [7]:
sgd(ssr_gradient, X, y, start=[0.5, 0.5], learn_rate=0.9, batch_size=3, n_iter=1_000, random_state=0)

array([4.90243287, 9.6332404 ])

In [8]:
print('sigma =', sigma, 'learn rate =', 0.9, 'N. iterations', 100)
gradient_descent(ssr_gradient, X, y, start=[0.5, 0.5], learn_rate=0.9,n_iter=100)

sigma = 10 learn rate = 0.9 N. iterations 100


array([-0.28617449, 10.57606435])

In [9]:
sgd(ssr_gradient, X, y, start=[0.5, 0.5], learn_rate=0.9, batch_size=3, n_iter=100, random_state=0)

array([ 0.6677476 , 13.37059579])

In [10]:
M = 16
X = np.random.rand(M,1)
y = theta0 + theta1 * X + sigma*np.random.randn(M,1)
print('sigma =', sigma, ', learn rate =', 0.9, ', N. iterations', 100, ', M=',M)
gradient_descent(ssr_gradient, X, y, start=[0.5, 0.5], learn_rate=0.9,n_iter=100)

sigma = 10 , learn rate = 0.9 , N. iterations 100 , M= 16


array([ 8.31175315, -1.72002086])

In [9]:
sgd(ssr_gradient, X, y, start=[0.5, 0.5], learn_rate=0.9, batch_size=3, n_iter=100, random_state=0)

array([2.90372616, 8.76262457])